***

**GDELT MongoDB**

***

Note: Syntax can be easily transitioned to Spark DataFrame.

We will do create 2 MongoDB collections:

- `gdelt_2019_mentions`: Mention tables merged with Events tables
- `gdelt_2019_events`: GKG tables but with string pseudo-lists transformed to lists

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pymongo as mb
plt.style.use('fivethirtyeight')
from pprint import pprint

# 1. GDELT Data Preprocessing with Pandas

**TODO in Spark with Scala**

1. gdelt_events:
    - Checks on variable types
2. gdelt_mentions:
    - Reduce DataFrame to 2 columns *GLOBALEVENTID* (for merge with gdelt_events) and *mentions* (the rest)
    - Checks on variable types
    - Create column for language of article / mention
3. gdelt_gkg:
    - Transfomring columns with poorly formated lists into dictionnaries
    - Checks on variable types

In [2]:
gdelt_events = pd.read_csv('test_local_gdelt_events.csv', index_col=0)
gdelt_mentions = pd.read_csv('test_local_gdelt_mentions.csv', index_col=0)
gdelt_gkg = pd.read_csv('test_local_gdelt_gkg.csv', index_col=0)

c:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
c:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
try:
    gdelt_mentions = gdelt_mentions.drop(['MentionDocOriginalLanguage'], axis=1)
except:
    pass

## 1.1. GDELT Events

In [4]:
%%time
# GDELT Events
gdelt_events_preprocessed_1 = gdelt_events.copy()
gdelt_events_preprocessed_1['ActionGeo_CountryCode'] = gdelt_events_preprocessed_1['ActionGeo_CountryCode'].fillna('unknown')
gdelt_events_preprocessed_1 = gdelt_events_preprocessed_1.fillna('')

gdelt_events_preprocessed_2 = gdelt_events_preprocessed_1[['GLOBALEVENTID']]
gdelt_events_preprocessed_2['events'] = gdelt_events_preprocessed_1 \
                                                .drop('GLOBALEVENTID', axis=1) \
                                                .to_dict(orient='records')

# Prepare Adequate Format for merge with GDELT Mentions
# gdelt_events_preprocessed_2 = gdelt_events_preprocessed_2.to_dict('records')
# pprint(gdelt_events_preprocessed_2[0])

Wall time: 4.77 s


c:\anaconda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [5]:
gdelt_events_preprocessed_2.head()

,GLOBALEVENTID,events
0,410412347,"{'SQLDATE': 20140218, 'MonthYear': 201402, 'Ye..."
1,410412348,"{'SQLDATE': 20140218, 'MonthYear': 201402, 'Ye..."
2,410412349,"{'SQLDATE': 20140218, 'MonthYear': 201402, 'Ye..."
3,410412350,"{'SQLDATE': 20140218, 'MonthYear': 201402, 'Ye..."
4,410412351,"{'SQLDATE': 20140218, 'MonthYear': 201402, 'Ye..."


## 1.2. GDELT Mentions

In [6]:
def get_article_mention_language(translateInfo):
    if translateInfo=='':
        language = 'eng'
    else:
        language = translateInfo.split(';', 1)[0][-3:]
    return language

gdelt_mentions['MentionDocTranslationInfo'] = gdelt_mentions['MentionDocTranslationInfo'].fillna('')

gdelt_mentions['MentionDocOriginalLanguage'] = \
                    gdelt_mentions['MentionDocTranslationInfo'].apply(lambda x: get_article_mention_language(x))

In [7]:
# gdelt_mentions['MentionDocOriginalLanguage'].value_counts()

In [8]:
gdelt_mentions_preprocessed = gdelt_mentions.merge(gdelt_events_preprocessed_2, on='GLOBALEVENTID', how='left')

In [9]:
print(gdelt_mentions.shape)
print(gdelt_mentions_preprocessed.shape)

(76937, 17)
(76937, 18)


In [10]:
# gdelt_mentions_processed_bis = gdelt_mentions_processed.groupby('GLOBALEVENTID')['mentions'].apply(list)
# gdelt_mentions_processed_bis = gdelt_mentions_processed_bis.reset_index()
gdelt_mentions_preprocessed[['GLOBALEVENTID', 'MentionDocOriginalLanguage', 'events']].head()

,GLOBALEVENTID,MentionDocOriginalLanguage,events
0,410412347,eng,"{'SQLDATE': 20140218, 'MonthYear': 201402, 'Ye..."
1,410412348,eng,"{'SQLDATE': 20140218, 'MonthYear': 201402, 'Ye..."
2,410412349,eng,"{'SQLDATE': 20140218, 'MonthYear': 201402, 'Ye..."
3,410412350,eng,"{'SQLDATE': 20140218, 'MonthYear': 201402, 'Ye..."
4,410412351,eng,"{'SQLDATE': 20140218, 'MonthYear': 201402, 'Ye..."


In [11]:
gdelt_gkg.transpose().iloc[:,0].to_dict()['AllNames'].split(';')

['Channel One,628',
 'Channel One,755',
 'Channel One,1449',
 'Channel One,1672',
 'Gen Aleksandr Lentsov,1994',
 'Eduard Basuryn,2123',
 'Ukrainian President Petro Poroshenko,2410',
 'Sergey Mikheyev,2552',
 'Hungarian Prime Minister Viktor Orban,2877',
 'Deputy Prime Minister Arkadiy Dvorkovich,3603',
 'Deputy Prime Minister Dvorkovich,3985',
 'Channel One,4835']

## 1.3. GDELT GKG

Columns to format from string to list :

- AllNames (;)
- Amounts (;)
- Counts (;)
- GCAM (,)
- Locations (;)
- Organizations (;)
- Persons (;)
- Quotations (;)
- Related Images
- Themes (;)
- V2Counts (;)
- **V2Locations** (;)
- V2Organizations (;)
- V2Persons (;)
- **V2Themes** (;)
- **V2Tone** (,)


In particular:

- **V2TONE**: (comma-delimited floating point numbers) This field contains a comma-delimited
list of six core emotional dimensions, described in more detail below. Each is recorded as a
single precision floating point number. This field is nearly identical in format and population as
the corresponding field in the GKG 1.0 format with the sole exception of adding the single new
WordCount variable at the end.

    1. **Tone**. (floating point number) This is the average “tone” of the document as a whole.
The score ranges from -100 (extremely negative) to +100 (extremely positive). Common
values range between -10 and +10, with 0 indicating neutral. This is calculated as
Positive Score minus Negative Score.
    2. **Positive Score**. (floating point number) This is the percentage of all words in the article
that were found to have a positive emotional connotation. Ranges from 0 to +100.
    3. **Negative Score**
    4. **Polarity**. (floating point number) This is the percentage of words that had matches in
the tonal dictionary as an indicator of how emotionally polarized or charged the text is.
If Polarity is high, but Tone is neutral, this suggests the text was highly emotionally
charged, but had roughly equivalent numbers of positively and negatively charged
emotional words.
    5. **Activity Reference Density**. (floating point number) This is the percentage of words
that were active words offering a very basic proxy of the overall “activeness” of the text
compared with a clinically descriptive text.
    6. **Self/Group Reference Density**. (floating point number) This is the percentage of all
words in the article that are pronouns, capturing a combination of self-references and
group-based discourse. News media material tends to have very low densities of such
language, but this can be used to distinguish certain classes of news media and certain
contexts.

- **Locations**: In cases where it is necessary to collapse by feature, the Geo_FeatureID column should be used, rather than the Geo_Fullname column. This is because the Geo_Fullname column captures the name of the location as expressed in the text and thus reflects differences in transliteration, alternative spellings, and alternative names for the same location. For example, Mecca is often spelled Makkah, while Jeddah is commonly spelled Jiddah or Jaddah. The Geo_Fullname column will reflect each of these different spellings, while the Geo_FeatureID column will resolve them all to the same unique GNS or GNIS feature identification number.

    1. **Location Type**
    2. **Location Fullname**
    3. **Location Country Code**
    4. **Location ADM1Code**
    5. **Location Latitude**
    6. **Location Longitude**
    7. **Location FeatureID** (text OR signed integer). This is the numeric GNS or GNIS FeatureID for this location OR a textual country or ADM1 code).
    
Example:
```python
['4',
 'Budapest, Budapest, Hungary',
 'RS',
 'HU05',
 '47.5',
 '19.0833',
 '-850553']
```

In [12]:
'TERROR;REBELS;TAX_ETHNICITY;TAX_ETHNICITY_UKRAINE'.split(';')

['TERROR', 'REBELS', 'TAX_ETHNICITY', 'TAX_ETHNICITY_UKRAINE']

In [13]:
gdelt_gkg_preprocessed = gdelt_gkg.fillna('')

In [14]:
gdelt_gkg_preprocessed.columns

Index(['GKGRECORDID', 'DATE', 'SourceCollectionIdentifier', 'SourceCommonName',
       'DocumentIdentifier', 'Counts', 'V2Counts', 'Themes', 'V2Themes',
       'Locations', 'V2Locations', 'Persons', 'V2Persons', 'Organizations',
       'V2Organizations', 'V2Tone', 'Dates', 'GCAM', 'SharingImage',
       'RelatedImages', 'SocialImageEmbeds', 'SocialVideoEmbeds', 'Quotations',
       'AllNames', 'Amounts', 'TranslationInfo', 'Extras'],
      dtype='object')

In [15]:
split_with_semicolon = ['AllNames', 'Amounts', 'Counts', 'Locations',
                        'Organizations', 'Persons', 'Quotations',
                        'RelatedImages', 'Themes', 'V2Counts',
                        'V2Locations', 'V2Organizations', 'V2Persons',
                        'V2Themes']

# for key_name in sorted(split_with_semicolon):
#     print(key_name)
#     print(gdelt_gkg_preprocessed.head(1)[key_name], '\n')
#     #row[key_name] = row[key_name].split(';')

In [16]:
def preprocess_gdelt_gkg_rows(row):
    """ AAA """
    # Split String Lists
    split_with_semicolon = ['AllNames', 'Amounts', 'Counts', 'Locations',
                            'Organizations', 'Persons', 'Quotations',
                            'RelatedImages', 'Themes', 'V2Counts',
                            'V2Locations', 'V2Organizations', 'V2Persons',
                            'V2Themes']
    
    split_with_comma = ['GCAM', 'V2Tone']
    
    # General Splits
    for key_name in split_with_semicolon:
        # Transform string into list
        row[key_name] = row[key_name].split(';')
        
        # Remove elements with empty strings
        row[key_name] = [el for el in row[key_name] if el != '']
    
    for key_name in split_with_comma:
        # Transform string into list
        row[key_name] = row[key_name].split(',')
        
        # Remove elements with empty strings
        row[key_name] = [el for el in row[key_name] if el != '']
    
    return row

def preprocess_gdelt_gkg_tone(x):
    res = dict()
    res['tone'] = float(x[0])
    res['positive_score'] = x[1]
    res['negative_score'] = x[2]
    res['polarity'] = x[3]
    res['activity_reference_density'] = x[4]
    res['group_reference_density'] = x[5]
    return res

def preprocess_gdelt_gkg_location(x):
    res_list = []
    # print(x)
    for location in x:
        x_split = location.split('#')
        res = dict()
        res['type'] = x_split[0]
        res['full_name'] = x_split[1]
        res['country_code'] = x_split[2]
        res['adm1_code'] = x_split[3]
        res['latitude'] = x_split[4]
        res['longitude'] = x_split[5]
        res['featureID'] = x_split[6]
        res_list.append(res)
        
    return res_list

In [17]:
# gdelt_gkg_preprocessed.head(5).transpose().sort_index()

In [18]:
# gdelt_gkg_preprocessed['AllNames'].head(1)

In [19]:
# gdelt_gkg_preprocessed_test['Locations'][0][0].split('#')

In [20]:
%%time
gdelt_gkg_preprocessed_test = gdelt_gkg_preprocessed.apply(lambda row: preprocess_gdelt_gkg_rows(row), axis=1)

Wall time: 1min 7s


In [21]:
# gdelt_gkg.isnull().sum()

In [22]:
%%time
gdelt_gkg_preprocessed_test['V2Tone'] = gdelt_gkg_preprocessed_test['V2Tone'].apply(lambda x: preprocess_gdelt_gkg_tone(x))

Wall time: 156 ms


In [23]:
%%time
gdelt_gkg_preprocessed_test['Locations'] = gdelt_gkg_preprocessed_test['Locations'].apply(lambda x: preprocess_gdelt_gkg_location(x))

Wall time: 402 ms


In [24]:
pprint(gdelt_gkg_preprocessed_test[['GKGRECORDID', 'Locations', 'Themes', 'V2Tone']].transpose().iloc[:,0].to_dict())

{'GKGRECORDID': '20150218230000-0',
 'Locations': [{'adm1_code': 'HU05',
                'country_code': 'RS',
                'featureID': '-850553',
                'full_name': 'Budapest, Budapest, Hungary',
                'latitude': '47.5',
                'longitude': '19.0833',
                'type': '4'},
               {'adm1_code': 'RS25',
                'country_code': 'RS',
                'featureID': '139814',
                'full_name': "Ogarevo, Kaluzhskaya Oblast', Russia",
                'latitude': '55.2031',
                'longitude': '35.8328',
                'type': '4'},
               {'adm1_code': 'HU',
                'country_code': 'RS',
                'featureID': 'HU',
                'full_name': 'Hungary',
                'latitude': '47',
                'longitude': '20',
                'type': '1'},
               {'adm1_code': 'RS55',
                'country_code': 'RS',
                'featureID': '-2993111',
                'full_name':

# 2. Analyzing GDELT with MongoDB

## 2.1. Bulk Insert Data into MongoDB

In [25]:
LOCAL_HOST_SERVER = 'localhost:27017'
mongo_client = mb.MongoClient(LOCAL_HOST_SERVER)

In [26]:
msbd_database = mongo_client['MSBD_2019_2020']

In [27]:
mb_gdelt_mentions = msbd_database['gdelt_mentions_events']
mb_gdelt_gkg = msbd_database['gdelt_gkg']

try:
    _ = mb_gdelt_mentions.drop()
    print('Collection dropped.')
except:
    print('Collection already dropped.')
    
try:
    _ = mb_gdelt_gkg.drop()
    print('Collection dropped.')
except:
    print('Collection already dropped.')
    
mb_gdelt_mentions = msbd_database['gdelt_mentions_events']
mb_gdelt_gkg = msbd_database['gdelt_gkg']

Collection dropped.
Collection dropped.


In [28]:
# gdelt_events_preprocessed['GLOBALEVENTID'].unique().shape[0] / gdelt_events_preprocessed.shape[0]

In [29]:
print(msbd_database.list_collection_names())

['enron_emails']


In [30]:
%%time
pandas_to_mongo_data = gdelt_mentions_preprocessed.copy()
pandas_to_mongo_data = pandas_to_mongo_data.fillna('')
# pandas_to_mongo_data = pandas_to_mongo_data.rename(columns={'GLOBALEVENTID': '_id'})
pandas_to_mongo_data = pandas_to_mongo_data.to_dict('records')

Wall time: 5.07 s


In [31]:
# pprint(pandas_to_mongo_data[0])

In [32]:
_ = mb_gdelt_mentions.insert_many(pandas_to_mongo_data)

In [33]:
%%time
pandas_to_mongo_data_gkg = gdelt_gkg_preprocessed_test.copy()
# pandas_to_mongo_data_gkg = pandas_to_mongo_data_gkg.rename(columns={'GKGRECORDID': '_id'})
pandas_to_mongo_data_gkg = pandas_to_mongo_data_gkg.to_dict('records')

Wall time: 3.51 s


In [34]:
%%time
_ = mb_gdelt_gkg.insert_many(pandas_to_mongo_data_gkg)

Wall time: 1min 20s


In [35]:
# for document in pandas_to_mongo_data_gkg:
#     _ = mb_gdelt_gkg.insert_one(document)

In [36]:
# gdelt_gkg_preprocessed_test[gdelt_gkg_preprocessed_test['GKGRECORDID'].duplicated(keep=False)].sort_values('GKGRECORDID')

In [37]:
# pprint(pandas_to_mongo_data_gkg[2982])

In [38]:
print(mb_gdelt_mentions.count_documents(filter={}))
print(mb_gdelt_gkg.count_documents(filter={}))

76937
36478


In [41]:
# gdelt_events_preprocessed[gdelt_events_preprocessed['GLOBALEVENTID']==410412359]['mentions'].tolist()

In [45]:
pprint(mb_gdelt_mentions.find_one({}))

{'ActionCharOffset': 4634,
 'Actor1CharOffset': -1,
 'Actor2CharOffset': 4594,
 'Confidence': 50,
 'EventTimeDate': 20150218230000,
 'Extras': '',
 'GLOBALEVENTID': 410412347,
 'InRawText': 1,
 'MentionDocLen': 6665,
 'MentionDocOriginalLanguage': 'eng',
 'MentionDocTone': -4.4776119402985,
 'MentionDocTranslationInfo': '',
 'MentionIdentifier': 'http://www.dailymaverick.co.za/article/2015-02-19-sona2015-jamming-state-security-agency-steps-forward-blames-low-ranking-operator/',
 'MentionSourceName': 'dailymaverick.co.za',
 'MentionTimeDate': 20150218230000,
 'MentionType': 1,
 'SentenceID': 19,
 '_id': ObjectId('5e26088acae47433c4415099'),
 'events': {'ActionGeo_ADM1Code': 'SF03',
            'ActionGeo_ADM2Code': '77359',
            'ActionGeo_CountryCode': 'SF',
            'ActionGeo_FeatureID': '-1299321',
            'ActionGeo_FullName': 'Waterkloof, Free State, South Africa',
            'ActionGeo_Lat': -30.3098,
            'ActionGeo_Long': 25.2971,
            'ActionGeo_Ty

In [46]:
pprint(mb_gdelt_gkg.find_one({})['V2Tone'])

{'activity_reference_density': '21.2566844919786',
 'group_reference_density': '0.53475935828877',
 'negative_score': '7.48663101604278',
 'polarity': '9.62566844919786',
 'positive_score': '2.13903743315508',
 'tone': -5.3475935828877}


## 2.2. GDELT Queries with MongoDB

- [More complex aggregations in MongoDB](https://stackoverflow.com/questions/43448389/mongo-aggregate-sum-of-values-in-a-list-of-dictionaries-for-all-documents)
- [Aggregate across multiple fields simultaneously](https://stackoverflow.com/questions/25843255/mongodb-aggregate-count-on-multiple-fields-simultaneously)

**1. Afficher le nombre d’articles/évènements qu’il y a eu pour chaque triplet (jour, pays de l’évènement, langue de l’article).**

In [44]:
# test_unwind_query = [
#     {'$unwind': '$mentions'},
#     {'$match': {'_id': 410412359}}
# ]

# test_unwind = list(gdelt_db.aggregate(test_unwind_query))
# test_unwind

In [62]:
test = [len(doc) for doc in test_unwind]

In [ ]:
result_events_triplet = list(gdelt_db.aggregate(query_mb_events_triplet))

In [26]:
query_mb_events_triplet = [
    {'$unwind': '$mentions'},
    {'$group': {'_id': {'date':'$SQLDATE',
                        'event_country': '$ActionGeo_CountryCode',
                        'article_language': '$mentions.MentionDocOriginalLanguage'},
                'NbEvents': {'$sum': 1}}
    }
]
pprint(query_mb_events_triplet)

[{'$unwind': '$mentions'},
 {'$group': {'NbEvents': {'$sum': 1},
             '_id': {'article_language': '$mentions.MentionDocOriginalLanguage',
                     'date': '$SQLDATE',
                     'event_country': '$ActionGeo_CountryCode'}}}]


In [27]:
result_events_triplet = list(gdelt_db.aggregate(query_mb_events_triplet))

In [28]:
len(result_events_triplet)

1428

In [34]:
pprint(result_events_triplet[:10])

[{'NbEvents': 2,
  '_id': {'article_language': 'rus', 'date': 20150217, 'event_country': 'RS'}},
 {'NbEvents': 67,
  '_id': {'article_language': 'eng', 'date': 20150218, 'event_country': 'GM'}},
 {'NbEvents': 60,
  '_id': {'article_language': 'spa', 'date': 20150219, 'event_country': 'SW'}},
 {'NbEvents': 4,
  '_id': {'article_language': 'lit', 'date': 20150218, 'event_country': 'LG'}},
 {'NbEvents': 127,
  '_id': {'article_language': 'ara', 'date': 20150219, 'event_country': 'SY'}},
 {'NbEvents': 5,
  '_id': {'article_language': 'spa', 'date': 20150218, 'event_country': 'ES'}},
 {'NbEvents': 5,
  '_id': {'article_language': 'eng', 'date': 20150219, 'event_country': 'LA'}},
 {'NbEvents': 3,
  '_id': {'article_language': 'ron', 'date': 20150218, 'event_country': 'US'}},
 {'NbEvents': 1,
  '_id': {'article_language': 'rus', 'date': 20150218, 'event_country': 'LH'}},
 {'NbEvents': 58,
  '_id': {'article_language': 'eng', 'date': 20150219, 'event_country': 'MY'}}]


**2. Pour un pays donné en paramètre, affichez les évènements qui y ont eu place triées par le nombre de mentions (tri décroissant); permettez une agrégation par jour/mois/année.**

**3. Pour une source de données passée en paramètre (gkg.SourceCommonName) affichez les thèmes, personnes, lieux dont les articles de cette sources parlent ainsi que le le nombre d’articles et le ton moyen des articles (pour chaque thème/personne/lieu); permettez une agrégation par jour/mois/année**

**4. Dresser la cartographie des relations entre les pays d’après le ton des articles : pour chaque paire (pays1, pays2), calculer le nombre d’article, le ton moyen (aggrégations sur Année/Mois/Jour, filtrage par pays ou carré de coordonnées).**